# Import

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.optimize import fsolve
from numpy import linalg

In [2]:
pi = np.pi

In [3]:
%matplotlib widget

# Pendule

Les constantes:

In [4]:
dt=0.005 #pas de temps pendule
L=1. #longueur pendule
g=10.  #gravite
c_amort=0.
T_fin=30.
n=int(T_fin/dt) #n=14 avec T_fin=3

Fonction exprimant les deux derivees:

In [5]:
def deriv_pendule(V, t): 
    return np.array([V[1],-c_amort*V[1]-g/L*np.sin(V[0])])

In [6]:
conditions_initiales_pendule = [pi/2, 0]

## Méthode d'Euler explicite

Vecteur de la solution [angle $\theta$, vitesse $\dot{\theta}$]

In [7]:
Sol = np.zeros([2,n]) 
Sol[:,0] = conditions_initiales_pendule

In [8]:
for k in range(n-1):
    t = k*dt
    Sol[:,k+1]=Sol[:,k]+dt*deriv_pendule(Sol[:,k], t)

In [9]:
fig, ax = plt.subplots(nrows=1, ncols=1, constrained_layout=True,figsize=(5,3.5))
x_pendule=np.linspace(0,T_fin,n)
ax.plot(x_pendule,Sol[0,:], '-', label='Angle')
ax.plot(x_pendule,Sol[1,:], '-', label='Vitesse')
ax.legend()
ax.set(title='Euler explicite', xlabel='time', ylabel=r'$\theta$'); ax.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Méthode de Heun (Euler Cauchy ou RK2)

In [10]:
Sol_RK2=np.zeros([2,n]) 
Sol_RK2[:,0]=conditions_initiales_pendule

In [11]:
for k in range(0,n-1):
    t = k*dt
    Sol_inter=Sol_RK2[:,k]+dt*deriv_pendule(Sol_RK2[:,k], t)
    Sol_RK2[:,k+1]=Sol_RK2[:,k]+0.5*dt*(deriv_pendule(Sol_RK2[:,k], t)+deriv_pendule(Sol_inter, t))

In [12]:
fig, ax = plt.subplots(nrows=1, ncols=1, constrained_layout=True,figsize=(5,3.5))
x_pendule=np.linspace(0,T_fin,n)
ax.plot(x_pendule,Sol_RK2[0,:], '-', label='Angle')
ax.plot(x_pendule,Sol_RK2[1,:], '-', label='Vitesse')
ax.legend()
ax.set(title='Heun', xlabel='time', ylabel=r'$\theta$'); ax.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Fonction de scipy odeint

In [13]:
t_ode = np.linspace(0,T_fin,n)

In [14]:
solution_ode = odeint(deriv_pendule, conditions_initiales_pendule, t_ode)

In [15]:
fig, ax = plt.subplots(nrows=1, ncols=1, constrained_layout=True,figsize=(5,3.5))
ax.plot(t_ode, solution_ode[:,0], label='Angle')
ax.plot(t_ode, solution_ode[:,1], label='Vitesse')
ax.legend()
ax.set(title='odeint', xlabel='time', ylabel=r'$\theta$'); ax.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Comparer les solutions

In [16]:
fig, ax = plt.subplots(nrows=1, ncols=1, constrained_layout=True,figsize=(5,3.5))
x_pendule=np.linspace(0,T_fin,n)
ax.plot(x_pendule,Sol[0,:], '-', label='Euler explicite')
ax.plot(x_pendule,Sol_RK2[0,:], '-', label='RK2')
ax.plot(t_ode, solution_ode[:,0], label='odeint')
ax.legend()
ax.set(title='Comparaison des méthodes', xlabel='time', ylabel=r'$\theta$'); ax.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Épidémie

In [17]:
a=0.02
b=0.5

In [18]:
Tps_final=15.
dt=0.02
N=int(Tps_final/dt)

In [19]:
def f_epidemie(X, t):
    Y=np.zeros(2)
    Y[0]=-a*X[0]*X[1]
    Y[1]=a*X[0]*X[1]-b*X[1]
    return Y

In [20]:
condtions_initiales_epidemie = [100.0,0.0001]
#condtions_initiales_epidemie = [50,50]

## Euler explicite

In [21]:
Euler = np.zeros([2,N+1])
Euler[:,0] = condtions_initiales_epidemie

In [22]:
for i in range(N):
    t = i*dt
    Euler[:,i+1]=Euler[:,i]+dt*f_epidemie(Euler[:,i], t)

In [23]:
print("taux finaux : Non contamines=", Euler[0,N],"contamines =",Euler[1,N])

taux finaux : Non contamines= 2.6959912874482823 contamines = 7.479057312703231


In [24]:
fig, ax = plt.subplots(nrows=1, ncols=1, constrained_layout=True,figsize=(5,3.5))
x=np.linspace(0,Tps_final,N+1)
ax.plot(x,Euler[0,:],label="Non-contaminés")
ax.plot(x,Euler[1,:],label="Malades")
ax.plot(x,Euler[0,:]+Euler[1,:],label="Total")
ax.set(title='Euler explicite', xlabel='temps', ylabel='N'); ax.grid()
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Méthode de Heun (RK2)

In [25]:
RK2=np.zeros([2,N+1])
RK2[:,0]=condtions_initiales_epidemie

In [26]:
for i in range(N):
    t = i*dt
    Sol_inter=RK2[:,i]+dt*f_epidemie(RK2[:,i],t)
    RK2[:,i+1]=RK2[:,i]+dt*0.5*(f_epidemie(Sol_inter,t)+f_epidemie(RK2[:,i],t))

In [27]:
print("taux finaux Heun : Non contamines=", RK2[0,N],"contamines =",[RK2[1,N]])

taux finaux Heun : Non contamines= 2.721132177808433 contamines = [7.173743463504104]


In [28]:
fig, ax = plt.subplots(nrows=1, ncols=1, constrained_layout=True,figsize=(5,3.5))
x=np.linspace(0,Tps_final,N+1)
ax.plot(x,RK2[0,:],label="Non-contaminés")
ax.plot(x,RK2[1,:],label="Malades")
ax.plot(x,RK2[0,:]+RK2[1,:],label="Total")
ax.set(title='RK2', xlabel='temps', ylabel='N'); ax.grid()
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## odeint

In [29]:
t_ode = np.linspace(0,Tps_final,int(Tps_final/dt))

In [30]:
Ode=odeint(f_epidemie,condtions_initiales_epidemie,t_ode)

In [31]:
fig, ax = plt.subplots(nrows=1, ncols=1, constrained_layout=True,figsize=(5,3.5))
x=np.linspace(0,Tps_final,N+1)
ax.plot(t_ode,Ode[:,0],label="Non-contaminés")
ax.plot(t_ode,Ode[:,1],label="Malades")
ax.plot(t_ode,Ode[:,0]+Ode[:,1],label="Total")
ax.set(title='odeint', xlabel='temps', ylabel='N'); ax.grid()
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Euler implicit

In [32]:
def f_impli(X,Xold):
    return X - dt*f_epidemie(X,t) - Xold

def jac_impli(X,Xold):
    return np.array([[a*X[1]*dt+1 , a*X[0]*dt],[ 1 -a*X[1]*dt +b*dt, 1+(b-a*X[0])*dt ]] )

In [33]:
Implicit=np.zeros([2,N+1])
Implicit[:,0]=condtions_initiales_epidemie
#Implicit[:,0]=[50,50]

In [34]:
# for i in range(N):
#     Implicit[:,i+1]=fsolve(f_impli,[50,50],args=Implicit[:,i],fprime=jac_impli)
#     #Implicit[:,i+1]=fsolve(f_impli,[50,50],args=Implicit[:,i])

In [35]:
for i in range(N):
    I=Implicit[:,i]
    residu=1
    niter=1
    #matrix=np.linalg.inv(jac_impli(I,Implicit[:,i]))
    while ((residu >1E-14) and (niter<10000)):
        niter=niter+1
        matrix=np.linalg.inv(jac_impli(I,Implicit[:,i]))
        I=I-np.dot(matrix,f_impli(I,Implicit[:,i]))
        residu=np.linalg.norm(f_impli(I,Implicit[:,i]))
    #print(linalg.cond(matrix, 'fro'))
    #print(linalg.eig(matrix))
    #print(niter)
    Implicit[:,i+1] = I

In [36]:
print("taux finaux Heun : Non contamines=", Implicit[0,N],"contamines =",[Implicit[1,N]])

taux finaux Heun : Non contamines= 2.7451714343897082 contamines = [6.8740279301929625]


In [37]:
fig, ax = plt.subplots(nrows=1, ncols=1, constrained_layout=True,figsize=(5,3.5))
x=np.linspace(0,Tps_final,N+1)
ax.plot(x,Implicit[0,:],label="Non-contaminés")
ax.plot(x,Implicit[1,:],label="Malades")
ax.plot(x,Implicit[0,:]+Implicit[1,:],label="Total")
ax.set(title='Implicit fsolve', xlabel='temps', ylabel='N'); ax.grid()
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Comparer les solutions

In [38]:
fig, ax = plt.subplots(nrows=1, ncols=1, constrained_layout=True,figsize=(5,3.5))
x=np.linspace(0,Tps_final,N+1)
ax.plot(x,Euler[0,:], '-', label='Euler explicite')
ax.plot(x,RK2[0,:], '-', label='RK2')
ax.plot(t_ode, Ode[:,0], label='odeint')
ax.plot(x, Implicit[0,:],'.-',  label='Euler implicite')
ax.legend()
ax.set(title='Comparaison des méthodes', xlabel='time', ylabel=r'$\theta$'); ax.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …